In [3]:
import numpy as np
import tools

NSCALE_points = 7
NSCALE_map = np.linspace(0, 3, NSCALE_points)

ZA_points = 7
air_masses_map = np.linspace(1, tools.calc_air_mass(75), ZA_points)
# ZA_map scaling with linear air_mass
ZA_map = tools.calc_zenith(air_masses_map)

freq_points = 240001
Tb_scalar_field = np.zeros((NSCALE_points, ZA_points, freq_points))
jacobian_scalar_field = np.zeros((NSCALE_points, ZA_points, freq_points))

# Load interpolation & jacobian data
for idx, NSCALE in enumerate(NSCALE_map):
    for jdx, AM in enumerate(air_masses_map):

        filename = f'MaunaKea_Tb_Spectrum_{AM:.1f}_{NSCALE:.1f}'
        data = np.load(f'{filename}.out')

        freq_map = data[:,0]
        
        # Second column is Tb in K
        Tb_scalar_field[idx,jdx] = data[:,2]

        # Jacobian
        conversation_factor = 2109.86 * 10**-3
        jacobian_scalar_field[idx,jdx] = (data[:,4] / conversation_factor)

In [5]:
from scipy.interpolate import CubicHermiteSpline

# Pass freq in points
interp_func = CubicHermiteSpline(x = (NSCALE_map[::2], ZA_map[::2], freq_map),
                                 y = Tb_scalar_field[::2,::2],
                                 dydx = jacobian_scalar_field[::2,::2])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.